In [5]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY=os.getenv("API_KEY")

In [33]:
query = {
      "q": '1 HLIANAT K _ ] "04N',
      "maxResults": 1,
      "orderBy": "relevance"
    }
response = requests.get('https://www.googleapis.com/books/v1/volumes/', params=query)



In [34]:
response_json = response.json()
book_info = response_json['items'][0]['volumeInfo']
book_info

{'title': 'Stress Tested: The Covid-19 Pandemic and Canadian National Security',
 'authors': ['Leah West'],
 'publisher': 'University of Calgary Press',
 'publishedDate': '2021-12',
 'description': "The emergence of COVID-19 has raised urgent and important questions about the role of Canadian intelligence and national security within a global health crisis. Some argue that the effects of COVID-19 on Canada represent an intelligence failure, or a failure of early warning. Others argue that the role of intelligence and national security in matters of health is--and should remain--limited. At the same time, traditional security threats have rapidly evolved, themselves impacted and influenced by the global pandemic. Stress Tested brings together leading experts to examine the role of Canada's national security and intelligence community in anticipating, responding to, and managing a global public welfare emergency. This interdisciplinary collection offers a clear-eyed view of successes, fa

In [32]:
title = book_info['title']
author = book_info['authors']
categories = book_info['categories']
print(f'Resolved Title: {title}')
print(f'Resolved Author: {author}')
print(f'Resolved categories: {categories}')

Resolved Title: Lobster Is the Best Medicine
Resolved Author: ['Liz Climo']
Resolved categories: ['Humor']
